In [1]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json
from openai import OpenAI

In [2]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        client = OpenAI(base_url="https://api.fanar.qa/v1",api_key=os.environ["FANAR_API_KEY"],)
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="azure"
deployment = APIKeyManager(model_type, "./azure.env")


In [3]:
class TopicClassifier:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def classify_topic(self, topic, information):
        """
        Classify podcast topic and determine optimal approach
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            
        Returns:
            JSON with classification results
        """
        
        prompt = f"""
أنت خبير في تحليل المواضيع وتصنيفها لإنتاج بودكاست عربي طبيعي وجذاب.

المهمة: حلل الموضوع التالي وحدد أفضل نهج لتقديمه في بودكاست عربي.

الموضوع: {topic}
المعلومات الأساسية: {information}

قم بتحليل الموضوع وإرجاع النتيجة بصيغة JSON تحتوي على:

{{
    "primary_category": "الفئة الرئيسية",
    "category_justification": "سبب اختيار هذه الفئة بناءً على طبيعة الموضوع",
    "optimal_style": "الأسلوب الأمثل للمناقشة",
    "discourse_pattern": "نمط الخطاب المناسب",
    "audience_engagement_goal": "هدف تفاعل الجمهور",
    "cultural_sensitivity_level": "مستوى الحساسية الثقافية",
    "controversy_potential": "احتمالية الجدل",
    "key_discussion_angles": [
        "زوايا النقاش الرئيسية المتوقعة",
        "النقاط التي ستثير اهتمام الجمهور العربي"
    ],
    "natural_tension_points": [
        "نقاط التوتر الطبيعية في الموضوع",
        "الجوانب التي قد تثير جدلاً صحياً"
    ],
    "cultural_connection_opportunities": [
        "فرص الربط بالثقافة العربية",
        "المراجع المحلية والإقليمية ذات الصلة"
    ]
}}

الفئات المتاحة:
1. "العلوم والتكنولوجيا" - للمواضيع التقنية والعلمية والابتكارات
2. "السياسة والشؤون العامة" - للمواضيع السياسية والأحداث الجارية والقضايا العامة
3. "القضايا الاجتماعية" - للمواضيع المجتمعية والعلاقات والقيم والتحديات الاجتماعية
4. "الرياضة والترفيه" - للمواضيع الرياضية والفنية والترفيهية
5. "التاريخ والثقافة" - للمواضيع التاريخية والتراثية والثقافية

الأساليب المتاحة:
- "حواري" - حوار طبيعي وودي بين المقدم والضيف
- "تعليمي" - تركيز على الشرح والتعليم بطريقة ممتعة
- "ترفيهي" - مرح وخفيف مع لمسات فكاهية
- "تحليلي" - نقاش عميق ومتخصص وتحليلي

أنماط الخطاب:
- "رسمي" - لغة رسمية ومحترمة
- "ودي" - لغة دافئة ومألوفة
- "جدلي" - نقاش حيوي مع وجهات نظر متعددة
- "سردي" - أسلوب حكواتي وقصصي



مستوى الحساسية الثقافية:
- "عالي" - يتطلب حذراً شديداً في التعامل
- "متوسط" - يحتاج مراعاة ثقافية معتدلة  
- "منخفض" - موضوع مقبول عموماً

احتمالية الجدل:
- "عالية" - موضوع مثير للجدل بطبيعته
- "متوسطة" - قد يثير بعض الاختلافات
- "منخفضة" - موضوع مقبول عموماً

تعليمات مهمة:
- حلل الموضوع بعمق وليس بشكل سطحي
- اعتبر السياق الثقافي العربي في التحليل
- ركز على ما يجعل الموضوع جذاباً للجمهور العربي
- تأكد أن التصنيف يخدم إنتاج محتوى طبيعي وتلقائي
- لا تضع علامات ```json في البداية أو النهاية
- أرجع JSON صحيح فقط بدون أي نص إضافي
-المدة 10 دقائق هي المدة المثلى للحلقة
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "أنت خبير في تحليل المواضيع وتصنيفها لإنتاج بودكاست عربي احترافي. تخصصك في فهم الجمهور العربي واهتماماته."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3  # Lower temperature for more consistent classification
        )
        
        return response.choices[0].message.content

   

In [4]:
# Testing Instructions:

# To test Step 1, add this to a new cell in your notebook:

# Test Step 1: Topic Classification
classifier = TopicClassifier(deployment, "gpt-4o")

# Test with the singlehood topic
topic = "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي"

information = '''
مع انتشار تقنيات الذكاء الاصطناعي بسرعة في العالم العربي، تزداد المخاوف حول تأثيرها على الهوية الثقافية واللغة العربية. 
تشير الدراسات إلى أن 78% من المحتوى الرقمي باللغة الإنجليزية، بينما المحتوى العربي لا يتجاوز 3%. 
معظم نماذج الذكاء الاصطناعي الحالية مدربة على بيانات غربية، مما يثير تساؤلات حول قدرتها على فهم السياق الثقافي العربي.
في المقابل، تسعى دول مثل الإمارات والسعودية لتطوير نماذج ذكاء اصطناعي عربية مثل "جايس" و"الحوراء" لمواجهة هذا التحدي.
التحدي الأكبر يكمن في كيفية الاستفادة من هذه التقنيات لتعزيز الثقافة العربية بدلاً من تهميشها، وضمان أن تخدم الذكاء الاصطناعي قيمنا ومبادئنا.
'''

# Run classification
classification_result = classifier.classify_topic(topic, information)
print("Classification Result:")
print(classification_result)



# Parse and examine the JSON
try:
    parsed_result = json.loads(classification_result)
    print(f"\nPrimary Category: {parsed_result['primary_category']}")
    print(f"Optimal Style: {parsed_result['optimal_style']}")
    print(f"Emotional Intent: {parsed_result['emotional_intent']}")
    print(f"Discourse Pattern: {parsed_result['discourse_pattern']}")
except:
    print("Error parsing JSON result")

Classification Result:
{
    "primary_category": "التاريخ والثقافة",
    "category_justification": "الموضوع يتناول الهوية الثقافية العربية وتأثير الذكاء الاصطناعي عليها، مما يجعله مرتبطاً بشكل مباشر بالثقافة والتراث العربي في العصر الرقمي.",
    "optimal_style": "تحليلي",
    "discourse_pattern": "ودي",
    "audience_engagement_goal": "إثارة الوعي حول أهمية الحفاظ على الهوية الثقافية العربية في ظل التطورات التقنية، وتحفيز الجمهور على التفكير النقدي حول دور الذكاء الاصطناعي في تشكيل المستقبل الثقافي.",
    "cultural_sensitivity_level": "عالي",
    "controversy_potential": "متوسطة",
    "key_discussion_angles": [
        "كيف يمكن للذكاء الاصطناعي أن يدعم اللغة العربية والثقافة بدلاً من تهميشها؟",
        "أهمية تطوير نماذج ذكاء اصطناعي عربية تلبي احتياجات المنطقة.",
        "دور الحكومات والمؤسسات العربية في تعزيز المحتوى الرقمي العربي.",
        "التحديات التي تواجه الذكاء الاصطناعي في فهم السياق الثقافي العربي."
    ],
    "natural_tension_points": [
        "الاعتماد المفرط على نماذج

In [5]:
class SimplePersonaGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_personas(self, topic, information, classification_result):
        """
        Generate simple but effective host and guest personas
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            
        Returns:
            JSON with simple host and guest personas
        """
        
        # Parse classification to understand the requirements
        try:
            classification = json.loads(classification_result)
        except:
            raise ValueError("Invalid classification JSON provided")
        
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        
        prompt = f"""
أنت خبير في تصميم شخصيات البودكاست العربي.

المهمة: أنشئ مقدم وضيف بسيطين ومناسبين لهذا الموضوع.

الموضوع: {topic}
المعلومات: {information}
الفئة: {primary_category}
الأسلوب المطلوب: {optimal_style}

أرجع النتيجة بصيغة JSON بسيط:

{{
    "host": {{
        "name": "اسم المقدم",
        "age": عمر رقمي,
        "background": "خلفية مختصرة في جملة واحدة",
        "personality": "وصف شخصيته في جملة واحدة",
        "speaking_style": "أسلوب حديثه في جملة واحدة"
    }},
    "guest": {{
        "name": "اسم الضيف", 
        "age": عمر رقمي,
        "background": "خلفية مختصرة في جملة واحدة",
        "expertise": "مجال خبرته في جملة واحدة",
        "personality": "وصف شخصيته في جملة واحدة",
        "speaking_style": "أسلوب حديثه في جملة واحدة"
    }},
    "why_good_match": "لماذا هذا المقدم والضيف مناسبان لهذا الموضوع - جملة واحدة"
}}

متطلبات:
- أسماء عربية مألوفة
- شخصيات بسيطة وقابلة للتصديق
- مناسبة للموضوع والأسلوب المطلوب
- المقدم فضولي والضيف خبير أو صاحب تجربة
- لا تضع علامات ```json
- أرجع JSON فقط
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تصميم شخصيات بودكاست بسيطة ومؤثرة. الأسلوب المطلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6
        )
        
        return response.choices[0].message.content

   

In [6]:
persona = SimplePersonaGenerator(deployment, "gpt-4o")
persona_result = persona.generate_personas(topic, information, classification_result)

In [7]:
class ContentOutlineGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_content_outline(self, topic, information, classification_result, personas_result):
        """
        Generate content-focused outline using classification and personas
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            personas_result: JSON string from Step 2 personas
            
        Returns:
            JSON with structured content outline
        """
        
        # Parse inputs
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided for classification or personas")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        discourse_pattern = classification.get("discourse_pattern", "")
        key_discussion_angles = classification.get("key_discussion_angles", [])
        natural_tension_points = classification.get("natural_tension_points", [])
        recommended_duration = classification.get("recommended_duration", "10 دقيقة")
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        host_background = personas.get("host", {}).get("background", "")
        guest_expertise = personas.get("guest", {}).get("expertise", "")
        
        prompt = f"""
أنت خبير في تصميم هيكل المحتوى للبودكاست العربي.

المهمة: أنشئ مخطط محتوى منطقي ومترابط لهذه الحلقة.

الموضوع: {topic}
المعلومات: {information}

السياق:
- الفئة: {primary_category}
- الأسلوب: {optimal_style}  
- نمط الخطاب: {discourse_pattern}
- المدة المقترحة: {recommended_duration}

الشخصيات:
- المقدم: {host_name} - {host_background}
- الضيف: {guest_name} - {guest_expertise}

زوايا النقاش المهمة: {key_discussion_angles}
نقاط التوتر الطبيعية: {natural_tension_points}

أرجع مخطط المحتوى بصيغة JSON:

{{
    "episode_info": {{
        "title": "عنوان الحلقة",
        "main_theme": "الموضوع الرئيسي في جملة واحدة",
        "target_duration": "المدة المتوقعة",
        "content_style": "نوع المحتوى المطلوب"
    }},
    "content_structure": {{
        "opening": {{
            "hook": "جملة افتتاحية جذابة محددة",
            "topic_introduction": "تقديم الموضوع بوضوح",
            "guest_introduction": "تقديم الضيف مع ربطه بالموضوع",
            "episode_roadmap": "خريطة ما سنناقشه في الحلقة"
        }},
        "main_sections": [
            {{
                "section_title": "عنوان القسم الأول",
                "main_points": [
                    "النقطة الأساسية الأولى",
                    "النقطة الأساسية الثانية", 
                    "النقطة الأساسية الثالثة"
                ],
                "key_questions": [
                    "سؤال رئيسي سيطرحه المقدم",
                    "سؤال متابعة مهم"
                ],
                "expected_insights": [
                    "رؤية متوقعة من الضيف",
                    "معلومة جديدة محتملة"
                ],
                "transition_to_next": "كيف الانتقال للقسم التالي"
            }},
            {{
                "section_title": "عنوان القسم الثاني", 
                "main_points": [
                    "النقطة الأساسية الأولى",
                    "النقطة الأساسية الثانية",
                    "النقطة الأساسية الثالثة"
                ],
                "key_questions": [
                    "سؤال رئيسي للقسم الثاني",
                    "سؤال استكشافي"
                ],
                "expected_insights": [
                    "رؤية متوقعة مختلفة",
                    "تحليل عميق"
                ],
                "transition_to_next": "كيف الانتقال للقسم التالي"
            }},
            {{
                "section_title": "عنوان القسم الثالث",
                "main_points": [
                    "النقطة الأساسية الأولى", 
                    "النقطة الأساسية الثانية",
                    "النقطة الأساسية الثالثة"
                ],
                "key_questions": [
                    "سؤال للقسم الثالث",
                    "سؤال تطبيقي"
                ],
                "expected_insights": [
                    "حلول أو توصيات",
                    "نصائح عملية"
                ],
                "transition_to_next": "كيف الانتقال للختام"
            }}
        ],
        "closing": {{
            "key_takeaways": [
                "الخلاصة الأولى المهمة",
                "الخلاصة الثانية المهمة", 
                "الخلاصة الثالثة المهمة"
            ],
            "final_message": "الرسالة الختامية الأساسية",
            "call_to_action": "دعوة للعمل أو التفكير",
            "next_steps": "خطوات يمكن للمستمع اتخاذها",
            "closing_gratitude": "شكر الضيف والمستمعين"
        }}
    }},
    "content_flow": {{
        "logical_progression": "كيف تتدفق الأفكار منطقياً",
        "narrative_arc": "القوس السردي للحلقة",
        "engagement_points": [
            "نقاط جذب انتباه المستمع",
            "لحظات تفاعل متوقعة"
        ],
        "complexity_progression": "كيف تتطور صعوبة المحتوى"
    }},
    "cultural_integration": {{
        "relevant_examples": [
            "أمثلة محلية مناسبة للموضوع",
            "حالات عربية ذات صلة"
        ],
        "cultural_sensitivity_notes": [
            "نقاط حساسة يجب مراعاتها",
            "توجيهات ثقافية"
        ],
        "regional_relevance": "كيف يرتبط الموضوع بالمنطقة العربية"
    }}
}}

متطلبات المحتوى:
- يجب أن يكون المحتوى مرتبطاً مباشرة بالموضوع: {topic}
- مناسب للأسلوب: {optimal_style}
- يراعي خلفية الشخصيات المحددة
- منطقي ومترابط ومناسب للمدة المقترحة
- يتضمن نقاط تفاعل طبيعية بين المقدم والضيف
- مناسب ثقافياً للجمهور العربي
- لا تضع علامات ```json 
- أرجع JSON صحيح فقط
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تصميم هيكل المحتوى للبودكاست العربي. تخصصك في بناء تدفق منطقي للأفكار يناسب الأسلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5  # Balanced for structured content
        )
        
        return response.choices[0].message.content

    

    def analyze_content_quality(self, outline_json):
        """
        Analyze the quality and coherence of the content outline
        """
        try:
            
            outline = json.loads(outline_json)
            
            analysis = {}
            
            # Check section count and balance
            main_sections = outline.get("content_structure", {}).get("main_sections", [])
            analysis["section_count"] = len(main_sections)
            analysis["balanced_sections"] = all(
                len(section.get("main_points", [])) >= 2 for section in main_sections
            )
            
            # Check content flow
            content_flow = outline.get("content_flow", {})
            analysis["has_narrative_arc"] = bool(content_flow.get("narrative_arc"))
            analysis["has_engagement_points"] = len(content_flow.get("engagement_points", [])) >= 2
            
            # Check cultural integration
            cultural = outline.get("cultural_integration", {})
            analysis["cultural_examples"] = len(cultural.get("relevant_examples", []))
            analysis["cultural_sensitivity"] = bool(cultural.get("cultural_sensitivity_notes"))
            
            # Overall quality score
            quality_indicators = [
                analysis["section_count"] >= 3,
                analysis["balanced_sections"],
                analysis["has_narrative_arc"], 
                analysis["has_engagement_points"],
                analysis["cultural_examples"] >= 2
            ]
            analysis["quality_score"] = sum(quality_indicators)
            analysis["quality_rating"] = "ممتاز" if analysis["quality_score"] >= 4 else "جيد" if analysis["quality_score"] >= 3 else "يحتاج تحسين"
            
            return analysis
            
        except:
            return {"error": "Could not analyze content quality"}



In [8]:
generator = ContentOutlineGenerator(deployment, "gpt-4o")
outline_result = generator.generate_content_outline(topic,information, classification_result, persona_result)

In [9]:
class SimpleStyleEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def enhance_with_category_style(self, outline_result, classification_result):
        """
        Add category-specific style enhancements to existing outline
        
        Args:
            outline_result: JSON string from Step 3 content outline
            classification_result: JSON string from Step 1 classification
            
        Returns:
            Original outline with added style enhancement keys
        """
        
        # Parse inputs
        try:
            original_outline = json.loads(outline_result)
            classification = json.loads(classification_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        
        prompt = f"""
أنت خبير في تطبيق أساليب مختلفة للبودكاست العربي.

المهمة: أضف تحسينات أسلوبية للمخطط الموجود بناءً على الفئة.

الفئة: {primary_category}
الأسلوب: {optimal_style}

المخطط الأصلي:
{outline_result}

إرشادات حسب الفئة:
- العلوم والتكنولوجيا: أضف شرح مبسط وأمثلة عملية
- السياسة والشؤون العامة: أضف وجهات نظر متعددة ونقاش متوازن  
- القضايا الاجتماعية: أضف قصص شخصية وتجارب حقيقية
- الرياضة والترفيه: أضف حماس وذكريات مثيرة
- التاريخ والثقافة: أضف أسلوب قصصي وربط بالحاضر

أرجع فقط التحسينات الإضافية بصيغة JSON:

{{
    "style_enhancements": {{
        "category_approach": "النهج العام لهذه الفئة",
        "opening_style_notes": "كيف تبدأ الحلقة بأسلوب الفئة",
        "question_style_guidance": "نوع الأسئلة المناسبة للفئة",
        "discussion_tone": "نبرة النقاش المطلوبة",
        "closing_style_notes": "كيف نختتم بأسلوب الفئة"
    }},
    "section_style_adjustments": [
        {{
            "section_index": 0,
            "style_approach": "كيف نتناول القسم الأول بأسلوب الفئة",
            "enhanced_questions": [
                "سؤال محسن لأسلوب الفئة",
                "سؤال متابعة مناسب"
            ],
            "tone_guidance": "نبرة هذا القسم"
        }},
        {{
            "section_index": 1,
            "style_approach": "منهج الفئة للقسم الثاني",
            "enhanced_questions": [
                "سؤال للقسم الثاني",
                "سؤال تحليلي"
            ],
            "tone_guidance": "نبرة القسم الثاني"
        }},
        {{
            "section_index": 2,
            "style_approach": "نهج الفئة للقسم الختامي",
            "enhanced_questions": [
                "سؤال ختامي",
                "سؤال للمستقبل"
            ],
            "tone_guidance": "نبرة الختام"
        }}
    ]
}}

متطلبات:
- أرجع فقط التحسينات وليس المخطط كاملاً
- طبق أسلوب الفئة {primary_category} فقط
- اجعل التحسينات بسيطة ومباشرة
- لا تضع علامات ```json
- أرجع JSON فقط
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تطبيق أساليب بسيطة للبودكاست. الفئة: {primary_category}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5
        )
        
        # Parse the style enhancements
        try:
            style_enhancements = json.loads(response.choices[0].message.content)
            
            # Add enhancements to original outline
            enhanced_outline = original_outline.copy()
            enhanced_outline.update(style_enhancements)
            
            return json.dumps(enhanced_outline, ensure_ascii=False, indent=2)
            
        except:
            # If parsing fails, return original outline as JSON string
            return json.dumps(original_outline, ensure_ascii=False, indent=2)

    def validate_style_enhancement(self, enhanced_outline_json):
        """
        Validate that style enhancements were added to original outline
        """
        required_enhancement_keys = ["style_enhancements", "section_style_adjustments"]
        style_required = ["category_approach", "opening_style_notes", "question_style_guidance", "discussion_tone"]
        
        try:
            
            outline = json.loads(enhanced_outline_json)
            
            missing_keys = []
            
            # Check that original structure still exists
            original_keys = ["episode_info", "content_structure", "content_flow", "cultural_integration"]
            for key in original_keys:
                if key not in outline:
                    missing_keys.append(f"original_{key}")
            
            # Check enhancement keys
            for key in required_enhancement_keys:
                if key not in outline:
                    missing_keys.append(key)
            
            # Check style enhancements structure
            if "style_enhancements" in outline:
                style_enh = outline["style_enhancements"]
                for key in style_required:
                    if key not in style_enh:
                        missing_keys.append(f"style_enhancements.{key}")
            
            # Check section adjustments
            if "section_style_adjustments" in outline:
                adjustments = outline["section_style_adjustments"]
                if not isinstance(adjustments, list) or len(adjustments) < 2:
                    missing_keys.append("section_style_adjustments (need at least 2)")
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Style enhancement validation successful - original outline preserved with additions"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

# Simple testing function
def test_simple_style_enhancement(deployment, outline_result, classification_result):
    """
    Test simple style enhancement that preserves original outline
    """
    enhancer = SimpleStyleEnhancer(deployment, "Fanar")
    
    # Generate style enhancement (adds to original outline)
    enhanced_result = enhancer.enhance_with_category_style(outline_result, classification_result)
    print("Enhanced Outline (Original + Style Enhancements):")
    print(enhanced_result)
    
    # Validate
    is_valid, message = enhancer.validate_style_enhancement(enhanced_result)
    print(f"\nValidation: {message}")
    
    # Show what was added
    if is_valid:
        try:
            
            enhanced = json.loads(enhanced_result)
            
            # Show original structure is preserved
            print(f"\n✅ Original Structure Preserved:")
            print(f"  - Episode Info: {'✓' if 'episode_info' in enhanced else '✗'}")
            print(f"  - Content Structure: {'✓' if 'content_structure' in enhanced else '✗'}")
            print(f"  - Content Flow: {'✓' if 'content_flow' in enhanced else '✗'}")
            
            # Show what enhancements were added
            if "style_enhancements" in enhanced:
                style_enh = enhanced["style_enhancements"]
                print(f"\n📝 Style Enhancements Added:")
                print(f"  Category Approach: {style_enh.get('category_approach', 'N/A')[:50]}...")
                print(f"  Question Style: {style_enh.get('question_style_guidance', 'N/A')[:50]}...")
            
            if "section_style_adjustments" in enhanced:
                adjustments = enhanced["section_style_adjustments"]
                print(f"\n🎯 Section Adjustments: {len(adjustments)} sections enhanced")
                
        except:
            print("Could not parse enhancement details")
    
    return enhanced_result

In [10]:
enhancer = SimpleStyleEnhancer(deployment, "gpt-4o")
enhanced_result = enhancer.enhance_with_category_style(outline_result, classification_result)

In [11]:
class FinalOutlineEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def add_spontaneity_and_culture(self, topic, information, classification_result, personas_result, enhanced_outline_result):
        """
        Add spontaneous elements and cultural integration to existing enhanced outline
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON from Step 1
            personas_result: JSON from Step 2  
            enhanced_outline_result: JSON from Step 4
            
        Returns:
            Complete outline with spontaneity and cultural elements added
        """
        
        # Parse inputs
        
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
            enhanced_outline = json.loads(enhanced_outline_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        cultural_sensitivity = classification.get("cultural_sensitivity_level", "")
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        host_personality = personas.get("host", {}).get("personality", "")
        guest_personality = personas.get("guest", {}).get("personality", "")
        
        prompt = f"""
أنت خبير في إضافة الحيوية والأصالة الثقافية للبودكاست العربي.

المهمة: أضف عناصر التلقائية والثقافة العربية للمخطط الموجود.

الموضوع: {topic}
المعلومات: {information}

السياق:
- الفئة: {primary_category}
- مستوى الحساسية الثقافية: {cultural_sensitivity}
- المقدم: {host_name} - {host_personality}
- الضيف: {guest_name} - {guest_personality}

المخطط المحسن الحالي:
{enhanced_outline_result}

أرجع فقط التحسينات الإضافية بصيغة JSON:

{{
    "spontaneity_enhancements": {{
        "opening_spontaneity": {{
            "natural_greeting": "ترحيب طبيعي يناسب شخصية المقدم",
            "spontaneous_hook": "بداية تلقائية جذابة", 
            "personality_touch": "لمسة شخصية وثقافية من المقدم"
        }},
        "section_spontaneity": [
            {{
                "natural_transitions": [
                    "انتقال طبيعي لبداية القسم",
                    "ربط تلقائي بالقسم السابق"
                ],
                "spontaneous_moments": [
                    "لحظة تفاعل تلقائي متوقعة",
                    "موقف قد يثير ضحكة أو دهشة"
                ],
                "personality_interactions": [
                    "كيف سيتفاعل المقدم بشخصيته",
                    "رد فعل متوقع من الضيف"
                ]
            }},
            {{
                "natural_transitions": [
                    "انتقال للقسم الثاني",
                    "ربط منطقي"
                ],
                "spontaneous_moments": [
                    "لحظة تفاعل للقسم الثاني",
                    "موقف عفوي محتمل"
                ],
                "personality_interactions": [
                    "تفاعل شخصيات في القسم الثاني",
                    "ديناميكية متوقعة"
                ]
            }},
            {{
                "natural_transitions": [
                    "انتقال للختام",
                    "تمهيد للخلاصة"
                ],
                "spontaneous_moments": [
                    "لحظة ختامية مؤثرة",
                    "تفاعل نهائي طبيعي"
                ],
                "personality_interactions": [
                    "تفاعل ختامي للشخصيات",
                    "لمسة إنسانية أخيرة"
                ]
            }}
        ],
        "closing_spontaneity": {{
            "spontaneous_conclusion": "ختام تلقائي وطبيعي",
            "personality_goodbye": "وداع شخصي وثقافي من المقدم والضيف",
            "audience_connection": "ربط أخير مع الجمهور"
        }}
    }},
    "cultural_integration": {{
        "section_cultural_elements": [
            {{
                "cultural_references": [
                    "مثل أو حكمة عربية مناسبة للقسم الأول",
                    "مرجع ثقافي محلي ذو صلة"
                ]
            }},
            {{
                "cultural_references": [
                    "مرجع ثقافي للقسم الثاني",
                    "تجربة عربية مشتركة"
                ]
            }},
            {{
                "cultural_references": [
                    "حكمة ختامية عربية",
                    "قول مأثور مناسب"
                ]
            }}
        ],
        "regional_touches": [
            "لمسات إقليمية مناسبة للموضوع",
            "مراجع محلية ذات صلة"
        ],
        "shared_experiences": [
            "تجارب عربية مشتركة",
            "ذكريات جماعية"
        ]
    }},
    "spontaneity_guide": {{
        "natural_fillers": [
            "حشو طبيعي مناسب للمقدم",
            "تعبيرات تلقائية للضيف"
        ],
        "reaction_triggers": [
            "ما قد يثير رد فعل طبيعي من المقدم",
            "ما قد يفاجئ الضيف ويجعله يتفاعل أو يقاطع"
        ]
    }}
}}

متطلبات مهمة:
- أرجع فقط التحسينات الجديدة وليس المخطط كاملاً
- كل العناصر يجب أن تكون مرتبطة بالموضوع: {topic}
- مناسبة لمستوى الحساسية الثقافية: {cultural_sensitivity}
- تعكس شخصيات المقدم والضيف المحددة
- طبيعية وغير مفتعلة
- أصيلة ثقافياً للجمهور العربي
- لا تضع علامات ```json
- أرجع JSON فقط
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في إضافة الحيوية والأصالة الثقافية للبودكاست العربي. الموضوع: {topic}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7  # Higher creativity for spontaneity
        )
        
        # Parse the final enhancements
        try:
            final_enhancements = json.loads(response.choices[0].message.content)
            
            # Add enhancements to existing enhanced outline
            complete_outline = enhanced_outline.copy()
            complete_outline.update(final_enhancements)
            
            return json.dumps(complete_outline, ensure_ascii=False, indent=2)
            
        except:
            # If parsing fails, return enhanced outline as JSON string
            return json.dumps(enhanced_outline, ensure_ascii=False, indent=2)

    def validate_final_outline(self, final_outline_json):
        """
        Validate that final enhancements were added to existing outline (simplified version)
        """
        required_enhancement_keys = ["spontaneity_enhancements", "cultural_integration", "spontaneity_guide"]
        
        try:
            outline = json.loads(final_outline_json)
            
            missing_keys = []
            
            # Check that original structure still exists (from Step 3 & 4)
            original_keys = ["episode_info", "content_structure", "content_flow", "style_enhancements"]
            for key in original_keys:
                if key not in outline:
                    missing_keys.append(f"original_{key}")
            
            # Check final enhancement keys
            for key in required_enhancement_keys:
                if key not in outline:
                    missing_keys.append(key)
            
            # Check spontaneity enhancements structure
            if "spontaneity_enhancements" in outline:
                spont_enh = outline["spontaneity_enhancements"]
                required_spont = ["opening_spontaneity", "section_spontaneity", "closing_spontaneity"]
                for key in required_spont:
                    if key not in spont_enh:
                        missing_keys.append(f"spontaneity_enhancements.{key}")
            
            # Check cultural integration (simplified)
            if "cultural_integration" in outline:
                cultural = outline["cultural_integration"]
                required_cultural = ["section_cultural_elements", "regional_touches", "shared_experiences"]
                for key in required_cultural:
                    if key not in cultural:
                        missing_keys.append(f"cultural_integration.{key}")
            
            # Check spontaneity guide (simplified)
            if "spontaneity_guide" in outline:
                guide = outline["spontaneity_guide"]
                required_guide = ["natural_fillers", "reaction_triggers"]
                for key in required_guide:
                    if key not in guide:
                        missing_keys.append(f"spontaneity_guide.{key}")
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Final outline validation successful - simplified structure with essential elements only"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

    def analyze_outline_completeness(self, final_outline_json):
        """
        Analyze the completeness and quality of the simplified final outline
        """
        try:
           
            outline = json.loads(final_outline_json)
            
            analysis = {}
            
            # Check that all previous steps are preserved
            analysis["step3_preserved"] = bool(outline.get("content_structure"))
            analysis["step4_preserved"] = bool(outline.get("style_enhancements"))
            
            # Check spontaneity guide (simplified)
            spontaneity_guide = outline.get("spontaneity_guide", {})
            analysis["spontaneity_guide_complete"] = (
                len(spontaneity_guide.get("natural_fillers", [])) >= 2 and
                len(spontaneity_guide.get("reaction_triggers", [])) >= 2
            )
            
            # Check cultural elements (simplified)
            cultural = outline.get("cultural_integration", {})
            analysis["cultural_elements_complete"] = (
                len(cultural.get("section_cultural_elements", [])) >= 3 and
                len(cultural.get("regional_touches", [])) >= 2 and
                len(cultural.get("shared_experiences", [])) >= 2
            )
            
            # Check spontaneity enhancements
            spont_enh = outline.get("spontaneity_enhancements", {})
            analysis["spontaneity_enhancements_complete"] = (
                bool(spont_enh.get("opening_spontaneity")) and
                bool(spont_enh.get("closing_spontaneity")) and
                len(spont_enh.get("section_spontaneity", [])) >= 3
            )
            
            # Overall readiness score
            readiness_indicators = [
                analysis["step3_preserved"],
                analysis["step4_preserved"],
                analysis["spontaneity_guide_complete"],
                analysis["cultural_elements_complete"],
                analysis["spontaneity_enhancements_complete"]
            ]
            analysis["readiness_score"] = sum(readiness_indicators)
            analysis["ready_for_dialogue"] = analysis["readiness_score"] >= 4
            analysis["pipeline_complete"] = analysis["readiness_score"] == 5
            
            return analysis
            
        except:
            return {"error": "Could not analyze outline completeness"}

# Testing function
def test_final_enhancement(deployment, topic, information, classification_result, personas_result, enhanced_outline_result):
    """
    Test final enhancement with spontaneity and cultural integration
    """
    enhancer = FinalOutlineEnhancer(deployment, "Fanar")
    
    # Generate final enhancement
    final_result = enhancer.add_spontaneity_and_culture(
        topic, information, classification_result, personas_result, enhanced_outline_result
    )
    print("Final Enhanced Outline:")
    print(final_result)
    
    # Validate
    is_valid, message = enhancer.validate_final_outline(final_result)
    print(f"\nValidation: {message}")
    
    # Analyze completeness
    if is_valid:
        completeness = enhancer.analyze_outline_completeness(final_result)
        print(f"\nCompleteness Analysis:")
        for key, value in completeness.items():
            print(f"  {key}: {value}")
        
        if completeness.get("ready_for_dialogue", False):
            print("\n✅ Outline is ready for dialogue generation!")
        else:
            print("\n⚠️ Outline may need additional refinement")
    
    return final_result

In [12]:
enhancer = FinalOutlineEnhancer(deployment, "gpt-4o")
final_result = enhancer.add_spontaneity_and_culture (topic, information, classification_result, persona_result, enhanced_result)
final_result

'{\n  "episode_info": {\n    "title": "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي",\n    "main_theme": "استكشاف تأثير الذكاء الاصطناعي على الهوية الثقافية العربية وكيفية الحفاظ عليها في ظل التحول الرقمي",\n    "target_duration": "10 دقيقة",\n    "content_style": "تحليلي بأسلوب ودي"\n  },\n  "content_structure": {\n    "opening": {\n      "hook": "مع انتشار الذكاء الاصطناعي في العالم العربي، هل نحن على أعتاب تغيير جذري في ثقافتنا وهويتنا؟",\n      "topic_introduction": "الذكاء الاصطناعي أصبح جزءاً أساسياً من حياتنا اليومية، لكن ما تأثيره على الهوية الثقافية العربية؟",\n      "guest_introduction": "معنا اليوم د. ليلى الحداد، خبيرة في تطوير نماذج ذكاء اصطناعي تراعي السياق الثقافي العربي، لتساعدنا في فهم هذا الموضوع المعقد.",\n      "episode_roadmap": "سنستعرض تأثير الذكاء الاصطناعي على الثقافة العربية، أهمية تطوير نماذج عربية، دور الحكومات والمؤسسات، وأهم التحديات التي تواجهنا."\n    },\n    "main_sections": [\n      {\n        "section_title": "القسم الأول: ت

In [13]:
class IterativeDialogueGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_opening_dialogue(self, final_outline_result, topic, personas_result):
        """
        Step 1: Generate opening dialogue (2-3 minutes)
        
        Args:
            final_outline_result: Complete outline from 5-step pipeline
            topic: Main topic
            personas_result: Host and guest personas
            
        Returns:
            Opening dialogue script
        """
        
        try:
            final_outline = json.loads(final_outline_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        # Extract key elements
        episode_info = final_outline.get("episode_info", {})
        content_structure = final_outline.get("content_structure", {})
        opening_info = content_structure.get("opening", {})
        spontaneity_enhancements = final_outline.get("spontaneity_enhancements", {})
        opening_spontaneity = spontaneity_enhancements.get("opening_spontaneity", {})
        cultural_integration = final_outline.get("cultural_integration", {})
        spontaneity_guide = final_outline.get("spontaneity_guide", {})
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        
        prompt = f"""
أنت خبير في كتابة الحوارات الطبيعية للبودكاست العربي.

المهمة: اكتب حوار الافتتاحية الطبيعي والتلقائي (2-3 دقائق فقط).

الموضوع: {topic}

الشخصيات:
- المقدم: {host.get('name', 'المقدم')} - {host.get('personality', '')}
- الضيف: {guest.get('name', 'الضيف')} - {guest.get('personality', '')}

عناصر الافتتاحية:
- الجملة الافتتاحية: {opening_info.get('hook', '')}
- تقديم الموضوع: {opening_info.get('topic_introduction', '')}
- تقديم الضيف: {opening_info.get('guest_introduction', '')}
- خريطة الحلقة: {opening_info.get('episode_roadmap', '')}

عناصر التلقائية:
- الترحيب الطبيعي: {opening_spontaneity.get('natural_greeting', '')}
- البداية التلقائية: {opening_spontaneity.get('spontaneous_hook', '')}
- اللمسة الشخصية: {opening_spontaneity.get('personality_touch', '')}

إرشادات الطبيعية:
- حشو طبيعي: {spontaneity_guide.get('natural_fillers', [])}
- محفزات التفاعل: {spontaneity_guide.get('reaction_triggers', [])}

متطلبات الحوار:
1. ابدأ بترحيب مختصر وطبيعي من المقدم
2. قدم الموضوع بطريقة جذابة وسريعة
3. قدم الضيف مع ربطه بالموضوع
4. أضف لمسة شخصية وثقافية مختصرة
5. انته بانتقال سريع وطبيعي للنقاش الرئيسي
6. استخدم 70% فصحى و 30% لمسات خليجية خفيفة
7. أضف حشو طبيعي قليل (يعني، طبعاً، اممم)
8. اجعل الحوار يبدو تلقائياً وسريعاً
9. ركز على الأساسيات - لا تطول كثيراً
10. اجعل النهاية تمهد مباشرة للنقاش الرئيسي

تنسيق الحوار:
المقدم: [النص]
الضيف: [النص]
المقدم: [النص]

أضف ملاحظات التفاعل عند الحاجة:
<laugh> للضحك
<pause: 2s> للتوقف
<overlap> للتداخل
<happy> للسعادة
<surprise> للمفاجأة

اكتب الحوار كاملاً:
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في كتابة حوارات بودكاست عربية طبيعية وتلقائية. المقدم: {host.get('name', 'المقدم')}، الضيف: {guest.get('name', 'الضيف')}"},
                {"role": "user", "content": prompt}
            ],
            temperature=1.2  # High creativity for natural dialogue
        )
        
        return response.choices[0].message.content

    def generate_main_discussion_dialogue(self, final_outline_result, topic, personas_result, opening_dialogue):
        """
        Step 2: Generate main discussion dialogue (6-7 minutes)
        
        Args:
            final_outline_result: Complete outline
            topic: Main topic
            personas_result: Personas
            opening_dialogue: Previous opening dialogue for context
            
        Returns:
            Main discussion dialogue script
        """
        
        try:
            final_outline = json.loads(final_outline_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        # Extract elements
        content_structure = final_outline.get("content_structure", {})
        main_sections = content_structure.get("main_sections", [])
        style_enhancements = final_outline.get("style_enhancements", {})
        spontaneity_enhancements = final_outline.get("spontaneity_enhancements", {})
        section_spontaneity = spontaneity_enhancements.get("section_spontaneity", [])
        cultural_integration = final_outline.get("cultural_integration", {})
        section_cultural_elements = cultural_integration.get("section_cultural_elements", [])
        spontaneity_guide = final_outline.get("spontaneity_guide", {})
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        
        # Build sections summary
        sections_summary = ""
        for i, section in enumerate(main_sections):
            sections_summary += f"""
القسم {i+1}: {section.get('section_title', '')}
- النقاط الأساسية: {section.get('main_points', [])}
- الأسئلة المهمة: {section.get('key_questions', [])}
- الرؤى المتوقعة: {section.get('expected_insights', [])}
"""
        
        # Build spontaneity elements
        spontaneity_summary = ""
        for i, spont in enumerate(section_spontaneity):
            spontaneity_summary += f"""
القسم {i+1} - العناصر التلقائية:
- انتقالات طبيعية: {spont.get('natural_transitions', [])}
- لحظات تلقائية: {spont.get('spontaneous_moments', [])}
- تفاعلات شخصية: {spont.get('personality_interactions', [])}
"""
        
        # Build cultural elements
        cultural_summary = ""
        for i, cultural in enumerate(section_cultural_elements):
            cultural_summary += f"""
القسم {i+1} - المراجع الثقافية: {cultural.get('cultural_references', [])}
"""
        
        prompt = f"""
أنت خبير في كتابة الحوارات الطبيعية للبودكاست العربي.

المهمة: اكتب حوار النقاش الرئيسي المركز (6-7 دقائق فقط).

الموضوع: {topic}

الشخصيات:
- المقدم: {host.get('name', 'المقدم')} - {host.get('personality', '')}
- الضيف: {guest.get('name', 'الضيف')} - {guest.get('personality', '')}

السياق - الحوار السابق (للاستمرارية):
{opening_dialogue[-500:]}  # آخر 500 حرف

أقسام النقاش المطلوب تغطيتها:
{sections_summary}

التحسينات الأسلوبية:
- نهج النقاش: {style_enhancements.get('question_style_guidance', '')}
- نبرة الحوار: {style_enhancements.get('discussion_tone', '')}

العناصر التلقائية لكل قسم:
{spontaneity_summary}

المراجع الثقافية لكل قسم:
{cultural_summary}

اللمسات الإقليمية: {cultural_integration.get('regional_touches', [])}
التجارب المشتركة: {cultural_integration.get('shared_experiences', [])}

إرشادات الطبيعية:
- حشو طبيعي: {spontaneity_guide.get('natural_fillers', [])}
- محفزات التفاعل: {spontaneity_guide.get('reaction_triggers', [])}

متطلبات الحوار:
1. ابدأ بانتقال طبيعي وسريع من الافتتاحية
2. غط النقاط الأساسية من الأقسام الثلاثة بتركيز
3. لا تلتزم بترتيب الأقسام - دع الحوار يتدفق بشكل مركز
4. ركز على النقاط الأهم والأسئلة الرئيسية فقط
5. أضف لحظة أو اثنتين من التفاعل التلقائي
6. استخدم مرجع ثقافي واحد بطبيعية
7. حافظ على شخصية المقدم والضيف
8. أضف لحظة اختلاف بسيطة أو نقاش
9. استخدم 70% فصحى و 30% لمسات خليجية
10. اجعل النقاش مكثفاً ومفيداً - تجنب الحشو الزائد
11. انته بتمهيد سريع للختام
12. لا تعيد الافتتاح مرة اخرة
13.اجعل الشرح بسيطاً ومفهوماً للجمهور العام

استخدم حشو المحادثة الطبيعي بكثافة متوسطة:
- تفكير: اممم، اههه، يعني كيف أقول، خلاص
- تأكيد: طبعاً، تماماً، بالضبط، صحيح
- تردد: بعنييييي، يعني، اه ما أدري، شوف
- انفعال: واو، يا الله، ما شاء الله، الله يعطيك العافية
- ربط: بس، لكن، وبعدين، يا أخي، اسمع
- خليجي خفيف: شلون، وش رايك، زين، ماشي الحال

تنسيق الحوار:
المقدم: [النص]
الضيف: [النص]

أضف ملاحظات التفاعل:
<laugh> <pause: 2s> <overlap> <happy> <surprise> <thinking>

اكتب الحوار كاملاً:
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في كتابة حوارات نقاش طبيعية ومتدفقة. تخصصك في جعل النقاش يبدو عفوياً رغم تغطية النقاط المهمة. الموضوع: {topic}"},
                {"role": "user", "content": prompt}
            ],
            temperature=1.2
        )
        
        return response.choices[0].message.content

    def generate_closing_dialogue(self, final_outline_result, topic, personas_result, opening_dialogue, main_discussion_dialogue):
        """
        Step 3: Generate closing dialogue (1-2 minutes)
        
        Args:
            final_outline_result: Complete outline
            topic: Main topic  
            personas_result: Personas
            opening_dialogue: Opening dialogue for context
            main_discussion_dialogue: Main discussion for context
            
        Returns:
            Closing dialogue script
        """
        
        
        try:
            final_outline = json.loads(final_outline_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        # Extract elements
        content_structure = final_outline.get("content_structure", {})
        closing_info = content_structure.get("closing", {})
        spontaneity_enhancements = final_outline.get("spontaneity_enhancements", {})
        closing_spontaneity = spontaneity_enhancements.get("closing_spontaneity", {})
        cultural_integration = final_outline.get("cultural_integration", {})
        spontaneity_guide = final_outline.get("spontaneity_guide", {})
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        
        prompt = f"""
أنت خبير في كتابة خواتيم الحوارات الطبيعية للبودكاست العربي.

المهمة: اكتب حوار الختام المختصر والمؤثر (1-2 دقيقة فقط).

الموضوع: {topic}

الشخصيات:
- المقدم: {host.get('name', 'المقدم')} - {host.get('personality', '')}
- الضيف: {guest.get('name', 'الضيف')} - {guest.get('personality', '')}

السياق - نهاية النقاش السابق (للاستمرارية):
{main_discussion_dialogue[-500:]}  # آخر 500 حرف

عناصر الختام المطلوبة:
- الخلاصات المهمة: {closing_info.get('key_takeaways', [])}
- الرسالة الختامية: {closing_info.get('final_message', '')}
- دعوة للعمل: {closing_info.get('call_to_action', '')}
- الخطوات التالية: {closing_info.get('next_steps', '')}
- شكر الضيف والمستمعين: {closing_info.get('closing_gratitude', '')}

العناصر التلقائية للختام:
- الختام التلقائي: {closing_spontaneity.get('spontaneous_conclusion', '')}
- الوداع الشخصي: {closing_spontaneity.get('personality_goodbye', '')}
- الربط مع الجمهور: {closing_spontaneity.get('audience_connection', '')}

العناصر الثقافية:
- اللمسات الإقليمية: {cultural_integration.get('regional_touches', [])}
- التجارب المشتركة: {cultural_integration.get('shared_experiences', [])}

إرشادات الطبيعية:
- حشو طبيعي: {spontaneity_guide.get('natural_fillers', [])}
- محفزات التفاعل: {spontaneity_guide.get('reaction_triggers', [])}

متطلبات الحوار:
1. ابدأ بانتقال طبيعي وسريع من النقاش
2. لخص النقاط المهمة بإيجاز شديد
3. أضف رسالة ختامية مختصرة ومؤثرة
4. اشكر الضيف بطريقة صادقة وسريعة
5. اشكر المستمعين وادعهم للتفاعل بإيجاز
6. أضف دعوة للعمل مختصرة وواضحة
7. اختتم بطريقة دافئة وسريعة
8. استخدم 70% فصحى و 30% لمسات خليجية
9. تجنب الإطالة - اجعله مركزاً ومؤثراً
10. اجعل النهاية تترك أثراً إيجابياً سريعاً
احذفْ الكلمات الزائدة؛ فالكتابة القوية موجزة. يجب أن تخلو الجملة من الكلمات غير الضرورية، والفقرة من الجمل غير الضرورية، للسبب نفسه الذي يجعل الرسم يخلو من الخطوط غير اللازمة والآلة من الأجزاء غير اللازمة. لا يعني هذا أن على الكاتب أن يجعل كل جمله قصيرة أو أن يتجنب التفاصيل ويكتفي بالخطوط العريضة، بل يعني أن يكون لكل كلمة أثر
تنسيق الحوار:
المقدم: [النص]
الضيف: [النص]

أضف ملاحظات التفاعل:
<grateful> <warm> <pause: 2s> <happy> <thoughtful>

اكتب الحوار كاملاً:
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في كتابة خواتيم مؤثرة وطبيعية للبودكاست العربي. تخصصك في ترك انطباع إيجابي دائم. الموضوع: {topic}"},
                {"role": "user", "content": prompt}
            ],
            temperature=1.2  # Slightly lower for more coherent conclusions
        )
        
        return response.choices[0].message.content

    def generate_complete_script(self, final_outline_result, topic, personas_result):
        """
        Generate complete podcast script using 3-step iterative approach
        
        Args:
            final_outline_result: Complete outline from 5-step pipeline
            topic: Main topic
            personas_result: Host and guest personas
            
        Returns:
            Dictionary with complete script parts and combined version
        """
        
        print("🎙️ بدء توليد السكريبت - 3 مراحل...")
        
        # Step 1: Opening
        print("\n📝 المرحلة 1: توليد الافتتاحية...")
        opening_dialogue = self.generate_opening_dialogue(final_outline_result, topic, personas_result)
        print("✅ تم إنجاز الافتتاحية")
        
        # Step 2: Main Discussion  
        print("\n📝 المرحلة 2: توليد النقاش الرئيسي...")
        main_discussion_dialogue = self.generate_main_discussion_dialogue(
            final_outline_result, topic, personas_result, opening_dialogue
        )
        print("✅ تم إنجاز النقاش الرئيسي")
        
        # Step 3: Closing
        print("\n📝 المرحلة 3: توليد الختام...")
        closing_dialogue = self.generate_closing_dialogue(
            final_outline_result, topic, personas_result, opening_dialogue, main_discussion_dialogue
        )
        print("✅ تم إنجاز الختام")
        
        # Combine all parts
        complete_script = f"""
=== الافتتاحية ===
{opening_dialogue}

=== النقاش الرئيسي ===
{main_discussion_dialogue}

=== الختام ===
{closing_dialogue}
"""
        
        print("\n🎉 تم إنجاز السكريبت الكامل!")
        
        return {
            "opening": opening_dialogue,
            "main_discussion": main_discussion_dialogue,
            "closing": closing_dialogue,
            "complete_script": complete_script,
            "script_length": len(complete_script),
            "estimated_duration": "10 دقائق تقريباً"
        }

    

# Testing function
def test_iterative_dialogue_generation(deployment, final_outline_result, topic, personas_result):
    """
    Test the complete 3-step iterative dialogue generation
    """
    generator = IterativeDialogueGenerator(deployment, "Fanar")
    
    # Generate complete script
    script_result = generator.generate_complete_script(final_outline_result, topic, personas_result)
    
    # Validate quality
    quality_analysis = generator.validate_script_quality(script_result)
    
    print(f"\n📊 تحليل جودة السكريبت:")
    for key, value in quality_analysis.items():
        print(f"  {key}: {value}")
    
    print(f"\n📈 معلومات السكريبت:")
    print(f"  طول السكريبت: {script_result['script_length']} حرف")
    print(f"  المدة المقدرة: {script_result['estimated_duration']}")
    print(f"  تقييم الجودة: {quality_analysis['quality_rating']}")
    
    # Show sample from each part
    print(f"\n📝 عينة من كل قسم:")
    print(f"الافتتاحية: {script_result['opening'][:200]}...")
    print(f"النقاش: {script_result['main_discussion'][:200]}...")
    print(f"الختام: {script_result['closing'][:200]}...")
    
    return script_result

In [14]:
diaglogue_generator  = IterativeDialogueGenerator(deployment)
print(diaglogue_generator.generate_complete_script(final_result, topic, persona_result))

🎙️ بدء توليد السكريبت - 3 مراحل...

📝 المرحلة 1: توليد الافتتاحية...
✅ تم إنجاز الافتتاحية

📝 المرحلة 2: توليد النقاش الرئيسي...
✅ تم إنجاز النقاش الرئيسي

📝 المرحلة 3: توليد الختام...
✅ تم إنجاز الختام

🎉 تم إنجاز السكريبت الكامل!
{'opening': "المقدم: السلام عليكم وأهلاً وسهلاً بجميع المستمعين الأعزاء! معكم سامي الجبري، واليوم صدقوني الموضوع اللي بنتكلم عنه شيّق وللغاية، وبيلامس حياتنا بشكل يومي تقريباً. تخيلوا معي ولو للحظة... مع انتشار الذكاء الاصطناعي في حياتنا، هل فعلاً ممكن يكون خطوة إيجابية للحفاظ على هويتنا العربية، ولا العكس؟ <pause: 2s> \n\nالذكاء الاصطناعي صار أساسي في كل شيء حوالينا، من التسوق، للتعليم، وحتى في كيفية تفاعلنا مع المحتوى على الإنترنت. لكن يبقى السؤال المحوري هنا: هل تأثيره على هويتنا وثقافتنا العربية إيجابي؟ وهل لازم نحذر من تأثيرات محتملة؟ <pause: 2s> \n\nبصراحة، هذا الموضوع أخدني لتفكير طويل. وأنتم، مثلما تعتقدون، أحب أبدأ بشي يجعلنا كلنا نشارك التفكير. هل الذكاء الاصطناعي فرصة ذهبية نغتنمها لتطوير المحتوى العربي وثقافتنا، أم هو تحدٍ خطير يمكن يعطل الجذور ا